<a href="https://colab.research.google.com/github/johanjun/kaggle_cassava_leaf/blob/main/Cassava_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library import

In [9]:
## library import
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os
import re
from PIL import Image
import shutil
from sklearn.model_selection import train_test_split
import random
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import xml.etree.ElementTree as et
import json
import pprint

# Data load

In [3]:
%cd /content/drive/MyDrive/Cassava_Leaf

/content/drive/MyDrive/Cassava_Leaf


In [4]:
os.getcwd()

'/content/drive/MyDrive/Cassava_Leaf'

In [5]:
## directory 설정
cur_dir = os.getcwd()
data_dir = os.path.join(cur_dir, 'data')
image_dir = os.path.join(data_dir, 'train_images')

In [6]:
## image file 수 확인
image_files = [fname for fname in os.listdir(image_dir) if os.path.splitext(fname)[-1] == '.jpg'] #확장자 분리
print(len(image_files))

21397


In [12]:
image_files[:10]

['787914923.jpg',
 '822641071.jpg',
 '810351903.jpg',
 '813744167.jpg',
 '821592762.jpg',
 '803715275.jpg',
 '816689138.jpg',
 '787415998.jpg',
 '813217011.jpg',
 '793206643.jpg']

In [11]:
with open(os.path.join(data_dir, 'label_num_to_disease_map.json')) as f:
    mapping = json.loads(f.read())
    pprint.pprint(mapping)

{'0': 'Cassava Bacterial Blight (CBB)',
 '1': 'Cassava Brown Streak Disease (CBSD)',
 '2': 'Cassava Green Mottle (CGM)',
 '3': 'Cassava Mosaic Disease (CMD)',
 '4': 'Healthy'}


In [13]:
#train, val
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

In [17]:
with open(os.path.join(data_dir, 'train.csv')) as f:
    df_train = pd.read_csv(f)

df_train.head()

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


## data split by label

In [30]:
df0 = df_train[df_train.label==0]
df0.label.value_counts()

0    1087
Name: label, dtype: int64

In [31]:
df1 = df_train[df_train.label==1]
df1.label.value_counts()

1    2189
Name: label, dtype: int64

In [32]:
df2 = df_train[df_train.label==2]
df2.label.value_counts()

2    2386
Name: label, dtype: int64

In [33]:
df3 = df_train[df_train.label==3]
df3.label.value_counts()

3    13158
Name: label, dtype: int64

In [36]:
df4 = df_train[df_train.label==4]
df4.label.value_counts()

4    2577
Name: label, dtype: int64

In [37]:
len(pd.concat([df0, df1, df2, df3, df4])) == len(df_train)

True